In [23]:
# Import Libraries and Packages
import pandas as pd
from tqdm import tqdm
import json

from bs4 import BeautifulSoup 
import requests  

pd.set_option("display.max_colwidth", None)

In [22]:
# Set Headers
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",
    }

In [39]:
def scrapArticleContent(url):
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.content, 'html.parser')
    article_content = soup.find_all("div", class_='entry clearfix')
    content_text = []
    for parag in article_content:
        sentences = parag.find_all('p')
        for sentence in sentences:
            text = sentence.get_text()
            content_text.append(text)
    return content_text

def scrapArticleLinkAndContent(page_num):
    web_url = f"https://umno-online.my/berita/rencana-suara-blogger/page/{page_num}/"
    response = requests.get(web_url, headers=HEADERS)
    soup = BeautifulSoup(response.content, 'html.parser')
    article_list = soup.find_all("div", class_='entry clearfix')
    data = []
    for article_tag in article_list:
        link = article_tag.find('a')
        if link:
            link_text = link.get('href')
            content = scrapArticleContent(link_text)
            data.append({'link': link_text, 'text': content})
    return data

def scrapAllArticles(start_page, end_page):
    all_data = []
    for page_num in tqdm(range(start_page, end_page + 1)):
        data = scrapArticleLinkAndContent(page_num)
        all_data.extend(data)
    return all_data



In [40]:
start_page = 1
end_page = 67
all_data = scrapAllArticles(start_page, end_page)

# Write data to JSON file
with open('article_data.json', 'w') as f:
    json.dump(all_data, f, indent=4)

print("Data scraped successfully and saved to article_data.json.")

100%|██████████| 67/67 [16:09<00:00, 14.47s/it]

Data scraped successfully and saved to article_data.json.
